In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
tf.enable_eager_execution()
tf.__version__

In [ ]:
cancer = datasets.load_breast_cancer()
cancer

In [ ]:
# key안의 value를 가져오는 방법
cancer['data']
cancer.get('data')			# key가 없으면 exception을 발생시키지 않고, None를 반환
cancer.data

In [ ]:
X = cancer.data
y = cancer.target
X, y

In [ ]:
X.dtype, y.dtype

In [ ]:
X = np.array(X, dtype='float32')
y = np.array(y, dtype='float32')
X, y

# y 종속변수
 - 0 -> 악성종양 - negative(관심없는 데이터)
 - 1 -> 양성종양 - positive(관심있는 데이터)
   > positive 데이터를 찾아야된다.

In [ ]:
y = y.reshape(-1, 1)
y

In [ ]:
y.shape

In [ ]:
y[0:30]

In [ ]:
# 양성, 악성 범주 스왑
np.where(y == 0, 1, 0)

In [ ]:
y = np.where(y == 0, 1, 0)

In [ ]:
y.reshape(569, 1)
y

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_test_split(X, y, test_size=0.2)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [ ]:
X_train, X_train.shape

In [ ]:
X_test, X_test.shape

In [ ]:
y_train, y_train.shape

In [ ]:
y_test, y_test.shape

In [ ]:
X.shape

In [ ]:
X[0:2]

In [ ]:
y_train, X_train

In [ ]:
import pandas as pd
import math

In [ ]:
df = pd.DataFrame({
					'price' : [950000000, 1050000000, 1000000000],
					'age' : [30, 36, 24],
					'sale' : [0, 1, 1]
				})

In [ ]:
df

In [ ]:
df.mean()

In [ ]:
df - df.mean()

In [ ]:
(df - df.mean()) ** 2

In [ ]:
((df - df.mean()) ** 2).sum()

In [ ]:
# 표준편차 / (data - 1) : 자유도라고 한다.
((df - df.mean()) ** 2).sum() / (len(df) - 1)

In [ ]:
df.var()

In [ ]:
(((df - df.mean()) ** 2).sum() / (len(df) - 1)) ** 0.5

In [ ]:
df.std()

In [ ]:
std = (((df - df.mean()) ** 2).sum() / len(df)) ** 0.5
std

In [ ]:
df

In [ ]:
dfX = df[['price', 'age']]
dfX

In [ ]:
# std를 구하기 위한 데이터 scaling
dfX - dfX.mean()

In [ ]:
(dfX - dfX.mean()) / std

In [ ]:
dfX_scale = (dfX - dfX.mean()) / std
dfX_scale

In [ ]:
(((dfX_scale - dfX_scale.mean()) ** 2).sum() / len(df)) ** 0.5

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
# fit(data_frame) : 각 칸의 평균, 표준편차 계산
scaler.fit(dfX)

In [ ]:
# transform(data_frame) : 각 칸 평균 빼고 표준편차 나눔
scaler.transform(dfX)

In [ ]:
# sigmoid를 이용해서 weight를 찾는 과정
# weight는 되도록 작은 값 -> 소수로 설정해야 답을 금방 찾는다

In [ ]:
np.random.seed(0)
np.random.randn(1, 30)		# 난수 리턴 (평균 0, 표준편차 1)

In [ ]:
# 0 ~ 1 사이 난수
# 난수의 확률이 1/n
# 주사위 각 숫자의 확률 1/6
tf.random.uniform([1, 30])

# 자비에르 초기화

In [ ]:
# idx = 1
# col = 30
줄의개수 = 1			# 어 이게 되네?
칸의개수 = 30

In [ ]:
줄칸평균 = (줄의개수 + 칸의개수) / 2
줄칸평균

In [ ]:
가중치범위 = math.sqrt(3.0 / 줄칸평균)
가중치범위

In [ ]:
tf.random_uniform([1, 30], -가중치범위, 가중치범위)

In [ ]:
initializer = tf.contrib.layers.xavier_initializer()

In [ ]:
initializer([1, 30])

In [ ]:
X_train

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)

In [ ]:
scaler.transform(X_train)

In [ ]:
initializer = tf.contrib.layers.xavier_initializer()

In [ ]:
w = initializer([30, 1])			# initializer([idx_cnt, col_cnt])
w

In [ ]:
w = tf.Variable(w)
w

In [ ]:
b = initializer([1])
b

In [ ]:
b = tf.Variable(initializer([1]))
b

In [ ]:
tf.matmul(X_train, w) + b

In [ ]:
hypothesis = tf.sigmoid(tf.matmul(X_train, w) + b)
hypothesis

In [ ]:
optimizer = tf.train.AdamOptimizer(0.0005)

In [ ]:
with tf.GradientTape() as tape :
	hypothesis = tf.sigmoid(tf.matmul(X_train, w) + b)
	cost = -tf.reduce_mean(y_train * tf.log(hypothesis) + (1 - y_train) * tf.log(1 - hypothesis))
	grads = tape.gradient(cost, [w, b])

	print('hypo = ', hypothesis[0:3])
	print('cost = ', cost)
	print('grad = ', grads)
	print('befo w = ', w)
	print('befo b = ', b)

optimizer.apply_gradients(grads_and_vars=zip(grads, [w, b]))
print('after w = ', w)
print('after b = ', b)
print('=' * 40)

In [ ]:
for step in range(1000) :
	with tf.GradientTape() as tape :
		hypothesis = tf.sigmoid(tf.matmul(X_train, w) + b)
		cost = -tf.reduce_mean(y_train * tf.log(hypothesis) + (1 - y_train) * tf.log(1 - hypothesis))
		grads = tape.gradient(cost, [w, b])

		print('hypo = ', hypothesis[0:3])
		print('cost = ', cost)
		print('grad = ', grads)
		print('before w = ', w)
		print('before b = ', b)

	optimizer.apply_gradients(grads_and_vars=zip(grads, [w, b]))
	print('after  w = ', w)
	print('after  b = ', b)
	print('=' * 40)